In [1]:
"""
7-8
10-23:
    - modified so that intervals can be found across all bpresses
"""

'\n7-8\n10-23:\n    - modified so that intervals can be found across all bpresses\n'

In [2]:
import json
#Pandas for saving datasets
import pandas as pd
#matplotlib for rendering
import matplotlib.pyplot as plt
#numpy for handeling matrix operations
import numpy as np
#time, to, well... keep track of time
import time
#iPython display for making sure we can render the frames
from IPython import display
#seaborn for rendering
import seaborn
import math
import os
import statistics as stat

# Find button presses / Counts 

In [3]:
## Reformats the button press array and returns the button press instances  ##
def org_bpress(arg):
    tem = len(arg.split())
    return [1] * tem
        

    
    

In [4]:
## Reformats the time array and returns the rounded (two decimals) time stamp  ##
## Timestamp represents the time the button was pressed, start of the run is t-zero ##
#returns an array of floats
import re
def org_tpress(arg):
    tem = arg.split()
    a=[]
    for i in tem:
        S = i
        pattern = re.compile(r'\-?\d+\.\d+')
        weights = list(map(float, re.findall(pattern, S)))
        a.append(round(float(weights[0]),2))
    #print(a)
    return(a)

    
    

In [5]:
## Returns the TR prior to the button press ## 
# arg is each time stamp: ([20.37, 45.95, 71.57, ...]) 
def tr_stam(arg):
    a = []
    for i in arg:
        a.append(int(round(i / 1.5,0)))
        #print(a)
    return(a)

In [6]:
## Returns a list of 209 TRs, 1 is a button press, 0 is not ## 
"""
all_TR_pres: 
 - iterate to num TRs (e.g. 210)
 - set the bpress TR (14,31,48) to 1, else the value is zero
 - returns: array of length num_trs with ones and zeros 
"""
def all_TR_pres(arg, num_trs):
    a = []
    for i in range(1,num_trs+1):
        if i in arg:
            a.append(1)
        else:
            a.append(0)
    return a
            

In [7]:
## Expand / Label TRs
"""
0 = SM
1 = SC
2 = OM
3 = OC
4 = Re
requires list of labels ouputed by psychopy (column 1 - MM_self_title.started, etc.)
returns label list (order is preserved) and TR labels
"""

def label_lists(label, num_tr):
    b = [[]]
    a = []
    for i in label:
        # substring label in psychopy output
        # if the first three characters == M_s, etc, then add correct indext to string
        if i[1:4] == "M_s":
            a.append("SM")
            b.append([0]*num_tr)
        elif i[1:4] == "C_s":
            a.append("SC")
            b.append([1]*num_tr)        
        elif i[1:4] == "M_o":
            a.append("OM")
            b.append([2]*num_tr)
        elif i[1:4] == "C_o":
            a.append("OC")
            b.append([3]*num_tr)     
        else:
            a.append("Re")
            b.append([4]*num_tr)     
    return a, b[1:]
 



In [8]:
def find_cond_index(sub_ses_labels):
    """
    For the array of ordered run names (i.e.'Re', 'SM',) find the two indexes per condition
    """ 
    lab_inx = []

    a = []
    b = []
    c = []
    d = []

    for i in enumerate(sub_ses_labels):
        if i[1] == "SM":
            # append the index according to where it appeared in the array
            a.append(i[0])
        if i[1] == "SC":
            b.append(i[0])
        if i[1] == "OM":
            c.append(i[0])
        if i[1] == "OC":
            d.append(i[0])

    # Create a dictionary where each key contains the appropriate indexes
    lab_indic = {
        'SM' : a,
        'SC' : b,
        'OM' : c,
        'OC' : d
    }
    return lab_indic 
    #np.vstack(lab_inx, ["SM", "SC", "OM", "OC"])

In [9]:
# Take in raw column data and return vector 0s and 1s, button press or no button press
"""
Input press data at e.g. sub.iloc[1,4]:
 - "['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']"
Input time data. This is the column immediately after :
 - '[20.37084077997133, 45.95158576697577, 71.56790153402835, ...] etc.
npress: reformat the above PRESS input data to integers
ts_stamp: reformat the above TIME input data into floats - time of runstart is t=0
tr_stam: take the reformatted data afrom ts_stamp ([20.37, 45.95, 71.57])
 - Divide timestamp, say 20.37, by 1.5. Result is 13.58. 
 - Round 13.58 up to 14
 - Between the 13th and 14th TR, the bpress occurs 
 - We round, so on the 14th TR the button press occurs 
 - Returns all of the TRs prior to the bpress: [14, 31, 48,..]
all_TR_pres: 
 - iterate to num TRs (e.g. 210)
 - set the bpress TR (14,31,48) to 1, else the value is zero
 - returns: array of length num_trs with ones and zeros 
"""
def run_press(pressN, timeN):
    n_press = org_bpress(pressN)
    ts_stamp = org_tpress(timeN)
    tr_stamp = tr_stam(ts_stamp)
    binar_210 = all_TR_pres(tr_stamp,num_trs)
    tr_210 = tr_stamp
    return binar_210, tr_210, ts_stamp

In [10]:
def bpress_arrays(dframe, sub_ses_labels, dtype):
    """
    Take in a column value (self or other) and count the number of presses for that particular run
    First get all self_btn info, add to an array. 
    Second, get all other info, add to an array
    All data is UNSORTED.
    args:
        cleaned subject dataframe
        dtype: tr_stamp = the tr when the button press occured
               ts_stamp = the time in seconds when the button press occured
               bin_arr = the binary array of when the button press occured (tr)
        
    return: 
        cond: unsorted array of labels (Self, then Other conds)
        Bpress: shape = 8 conditions by X TRs. 1s and 0s


    """
    if dtype == "tr_stamp":
        get_press = 1
    elif dtype == "ts_stamp":
        get_press= 2
    elif dtype == "bin_stamp":
        get_press= 0
    else:
        print("enter dtype arg correctly yo")
    
    df = []
    cond = []
    
    for col_ind, col_val in enumerate(dframe.columns):
        if col_val == "self_btn.keys":
            # For all Rows in the self column
            # if the row is of type string
            for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                #if type(row_val) is type("a"):
                if row_val == "None":
                    print("hi")
                    df.append(np.array([]))
                    cond.append(sub_ses_labels[row_inx])
                elif row_val != "-1":
                    # Add to condition array
                    # There are 10 rows, and 10 runs. 
                    # I could pull from the third column to get the string name
                    # Instead i pull from the label order in sub_ses, which corresponds to the
                    # row_inx
                    cond.append(sub_ses_labels[row_inx])
                    """
                    run_press takes in the button presses, then the timing info (+1)
                    returns: a list of button presses, 1s and 0s for each run - [0]
                            - OR the TR when the button press occured - [1]
                    """
                    print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                    bpress_info = dframe.iloc[row_inx,col_ind]
                    time_info = dframe.iloc[row_inx,col_ind+1]
                    df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                    tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                    print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
        elif col_val == "other_j_btn1.keys" or col_val =="other_n_btn.keys":
            if sub == 'sub-000' and col_val =="other_n_btn.keys":
                print('hi')
                df.append(np.array([]))
                df.append(np.array([]))
                cond.append('OM')
                cond.append('OC')
            else:
                for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                    if row_val == "None":
                        print("hi")
                        df.append(np.array([]))
                        cond.append(sub_ses_labels[row_inx])
                    elif row_val != "-1":
                        print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                        # Add to condition array
                        cond.append(sub_ses_labels[row_inx])
                        # append button press info
                        bpress_info = dframe.iloc[row_inx,col_ind]
                        time_info = dframe.iloc[row_inx,col_ind+1]
                        df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                        tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                        print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
    return np.asarray(df), cond



In [11]:
def sort_bpress_data(unsort_bpress,lab_indic_inc, lab_indic_cor):
    """
    1. find the correct index for SM, SC, etc.
    2. find the index of SM in the other dataframe
    3. set the empyt dataframe index of the correct SM to the data of the unsort
    4. Iterate
    """
    # create empty dframe for organized bpress data from shape of unsorted dframe
    sort_bpress = np.zeros(unsort_bpress.shape)
    print("Finished!")
    for i in range(0,2):
        sort_bpress[lab_indic_cor['SM'][i]] = unsort_bpress[lab_indic_inc['SM'][i]]
        sort_bpress[lab_indic_cor['SC'][i]] = unsort_bpress[lab_indic_inc['SC'][i]]
        sort_bpress[lab_indic_cor['OM'][i]] = unsort_bpress[lab_indic_inc['OM'][i]]
        sort_bpress[lab_indic_cor['OC'][i]] = unsort_bpress[lab_indic_inc['OC'][i]]
    return sort_bpress 
    
    
    
    
    
    

In [12]:
def sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor):
    """
    1. find the correct index for SM, SC, etc.
    2. find the index of SM in the other dataframe
    3. set the empyt dataframe index of the correct SM to the data of the unsort
    4. Iterate
    """
    # create empty dframe for organized bpress data from shape of unsorted dframe
    #sort_bpress = np.zeros(unsort_bpress.shape)
    onsets = [None] * len(unsort_bpress)
    
    for i in range(0,2):
        onsets[lab_indic_cor['SM'][i]] = unsort_bpress[lab_indic_inc['SM'][i]]
        onsets[lab_indic_cor['SC'][i]] = unsort_bpress[lab_indic_inc['SC'][i]]
        onsets[lab_indic_cor['OM'][i]] = unsort_bpress[lab_indic_inc['OM'][i]]
        onsets[lab_indic_cor['OC'][i]] = unsort_bpress[lab_indic_inc['OC'][i]]
    return onsets 
    
    
    
    
    
    

# Begin

In [13]:
behav_p = '/jukebox/graziano/coolCatIsaac/ATM/data/behavioral'
sav_work = "/jukebox/graziano/coolCatIsaac/ATM/data/work/results/"
sav_bpress = "/jukebox/graziano/coolCatIsaac/ATM/data/work/results/bpress_GLM/behav"
sub_list = ["sub-001","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-011","sub-012","sub-013","sub-014"]
sub_list = ["sub-000","sub-001", "sub-002","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-010","sub-011","sub-012","sub-013","sub-014","sub-015", "sub-016","sub-017", 
            "sub-018","sub-019", "sub-020","sub-021"]

sub_list = ["sub-000","sub-001", "sub-002","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-010","sub-011","sub-012","sub-013","sub-014","sub-015", "sub-016","sub-017", 
            "sub-018","sub-019", "sub-020","sub-021","sub-022","sub-023","sub-024","sub-025","sub-026", "sub-027"]

sub_list = ["sub-000","sub-001", "sub-002","sub-003","sub-004","sub-005","sub-006","sub-007","sub-008","sub-009",
            "sub-010","sub-011","sub-012","sub-013","sub-014","sub-015", "sub-016","sub-017", 
            "sub-018","sub-019", "sub-020","sub-021","sub-022","sub-023","sub-024","sub-025","sub-026", "sub-027"]

#sub_list = ["test"]

# No sub-000 or sub-019 and an issue with sub-009! can you believe it
"""
Issues:
- sub-000: no button presses for other_n_monitor. values are 1.0 instead of "-1.0". Fucks errthang up **SOLVED
- sub-019: no button presses for other_n_monitor. Values is None, instead of "-1.0". Fucks errthang up **SOLVED
    * only one button press for other count! check effort... may be shit subject
- sub-009 only has one button press for SM **Not an issue for button press counts - just interval calculations
- sub-024 No button press for last self monitor condition! no eyetracking, maybe asleep! maybe should delete... ugh
"""
#sub_list = ['sub-015']
num_trs = 209

In [14]:
#sub_list = ['sub-021']
#sub_list = ['sub-026']

# SUM bpress per run# 

In [15]:
cond_total_bpress = {
    'SM': [],
    'SC': [],
    'OM': [],
    'OC': []
}
cond_elim_bpress = {
    'SM': [],
    'SC': [],
    'OM': [],
    'OC': []
}
new_bpress_dat = {}

interval = 12

for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    if sub == 'sub-002': behav = behav.fillna("-1")
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "ts_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor)   



    cond_bpress_dat = {}
    for lab in lab_indic_cor:
        print(lab, "differences are:")
        # mean / vars dict
        
        run_bpress_dat = []
        
        count_tot_bpress = []
        count_elim = []
        for run in range(2):
            
            print('bpress',sort_bpress[lab_indic_cor[lab][run]])
            run_bpress_arr = sort_bpress[lab_indic_cor[lab][run]]
            

            # Insert zero for the start of the task
            #run_bpress_arr = np.insert(run_bpress_arr,0,0) 
            try:
                difs = np.zeros(len(run_bpress_arr)-1)
                
            except:
                run_bpress_dat.append(np.array([]))
                continue
            for idx in range(1,len(run_bpress_arr)):
                # set the current index of the empty array (diffs) to the subtracted value
                # of the current bpress idx minus the bpress -1 index
                difs[idx-1] = run_bpress_arr[idx] - run_bpress_arr[idx-1]

            add_press = []
            add_press.append(run_bpress_arr[0])
            last = False
            
            for idx, dif in enumerate(difs):
                #print(run_bpress_arr[idx])
                if dif < interval: ### *** if different is less than 12 second 
                    if last == True:
                        # add that bpress
                        add_press.append(run_bpress_arr[idx+1])
                        # last remains true
                    else:
                        last=True
                        continue
                else:
                    add_press.append(run_bpress_arr[idx+1])
                    last=False
            run_bpress_dat.append(np.asarray(add_press))
            print('addpress:',add_press)
            print('difs',difs)
            print('num eliminated:', len(run_bpress_arr)-len(add_press), 'of', len(run_bpress_arr))
            count_tot_bpress.append(len(run_bpress_arr))
            count_elim.append(len(add_press))
        #break
        
        cond_total_bpress[lab].append(np.sum(np.asarray(count_tot_bpress)))
        cond_elim_bpress[lab].append(np.sum(np.asarray(count_elim)))
        cond_bpress_dat[lab] = run_bpress_dat
    new_bpress_dat[sub] = cond_bpress_dat
        
    
print('el fin biotches')

BEGIN:  sub-000
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  14
hi
SM differences are:
bpress [ 94.6  115.6  125.55 154.02 221.83 269.04]
addpress: [94.6, 115.6, 154.02, 221.83, 269.04]
difs [21.    9.95 28.47 67.81 47.21]
num eliminated: 1 of 6
bpress [ 27.81  59.33  74.99 103.14 126.75 154.29 167.91 190.39 208.15 225.51
 251.94 276.34 299.11]
addpress: [27.81, 59.33, 74.99, 103.14, 126.75, 154.29, 167.91, 1

/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


 Sum bpress:  8
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  12
rowval: 5 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  9
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  8
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  20
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 6 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 7 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  13
SM differences are:
bpress [  8.53  53.51 106.1  136.66 185.53 209.58 224.18 301.88]
addpress: [8.53, 53.51, 106.1, 136.66, 185.53, 209.58, 224.18, 301.88]
difs [44.98 52.59 30.56 

rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  17
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  12
rowval: 6 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  11
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  24
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  16
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  29
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 8 colval: 10

num eliminated: 0 of 1
SC differences are:
bpress [ 16.26  47.62  82.82 116.9  180.35 209.74 235.32 261.82 292.33]
addpress: [16.26, 47.62, 82.82, 116.9, 180.35, 209.74, 235.32, 261.82, 292.33]
difs [31.36 35.2  34.08 63.45 29.39 25.58 26.5  30.51]
num eliminated: 0 of 9
bpress [ 17.61  37.44  47.89  67.8   86.62 105.22 125.22 147.62 160.42 189.5
 226.45 262.21 275.92 303.08]
addpress: [17.61, 37.44, 67.8, 86.62, 105.22, 125.22, 147.62, 160.42, 189.5, 226.45, 262.21, 275.92, 303.08]
difs [19.83 10.45 19.91 18.82 18.6  20.   22.4  12.8  29.08 36.95 35.76 13.71
 27.16]
num eliminated: 1 of 14
OM differences are:
bpress [209.78]
addpress: [209.78]
difs []
num eliminated: 0 of 1
bpress [197.03]
addpress: [197.03]
difs []
num eliminated: 0 of 1
OC differences are:
bpress [ 17.75  39.31  62.36  77.5   98.88 113.43 133.94 174.74 193.18 212.58
 240.68 269.13 291.73 307.24]
addpress: [17.75, 39.31, 62.36, 77.5, 98.88, 113.43, 133.94, 174.74, 193.18, 212.58, 240.68, 269.13, 291.73, 307.24]
difs 

rowval: 1 colval: 4 dframe val: ['1']
SM Sum bpress:  1
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  14
rowval: 6 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  14
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  8
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  11
rowval: 4 colval: 7 dframe val: ['1', '1', '1']
OM Sum bpress:  3
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  14
rowval: 7 colval: 10 dframe val: ['1', '1', '1', '1', '1']
OM Sum bpress:  5
SM differences are:
bpress [129.91]
addpress: [129.91]
difs []
num eliminated: 0 of 1
bpress [ 57.96  88.03 129.75 166.09 194.74 238.75 263.3  289.58]
addpress: [57.96, 88.03, 129.75, 166.09, 194.74, 238.75, 263.3, 289.58]
difs [30.07 4

num eliminated: 0 of 14
OM differences are:
bpress []
bpress [ 92.33 115.91 126.83 159.23 180.1  238.2 ]
addpress: [92.33, 115.91, 159.23, 180.1, 238.2]
difs [23.58 10.92 32.4  20.87 58.1 ]
num eliminated: 1 of 6
OC differences are:
bpress [ 20.39  40.81  69.66  89.48 106.96 135.63 187.72 209.71 235.99 261.37
 280.54 298.96]
addpress: [20.39, 40.81, 69.66, 89.48, 106.96, 135.63, 187.72, 209.71, 235.99, 261.37, 280.54, 298.96]
difs [20.42 28.85 19.82 17.48 28.67 52.09 21.99 26.28 25.38 19.17 18.42]
num eliminated: 0 of 12
bpress [154.95]
addpress: [154.95]
difs []
num eliminated: 0 of 1
BEGIN:  sub-020
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  14
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  11
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1

OM Sum bpress:  5
SM differences are:
bpress [117.56 172.6  222.47 245.77]
addpress: [117.56, 172.6, 222.47, 245.77]
difs [55.04 49.87 23.3 ]
num eliminated: 0 of 4
bpress []
SC differences are:
bpress [ 34.55  66.88  95.75 120.6  162.12 254.99]
addpress: [34.55, 66.88, 95.75, 120.6, 162.12, 254.99]
difs [32.33 28.87 24.85 41.52 92.87]
num eliminated: 0 of 6
bpress [ 30.66  55.78  82.73 113.48 141.77 169.02 199.54 226.49 256.35 283.8 ]
addpress: [30.66, 55.78, 82.73, 113.48, 141.77, 169.02, 199.54, 226.49, 256.35, 283.8]
difs [25.12 26.95 30.75 28.29 27.25 30.52 26.95 29.86 27.45]
num eliminated: 0 of 10
OM differences are:
bpress [ 22.19  43.4   85.78 193.57]
addpress: [22.19, 43.4, 85.78, 193.57]
difs [ 21.21  42.38 107.79]
num eliminated: 0 of 4
bpress [ 85.47 119.15 150.18 199.72 250.23]
addpress: [85.47, 119.15, 150.18, 199.72, 250.23]
difs [33.68 31.03 49.54 50.51]
num eliminated: 0 of 5
OC differences are:
bpress [ 18.1   40.53  62.5   83.01 104.36 125.59 142.86 169.93 190.26 21

In [16]:
cond_elim_bpress

{'SM': [18,
  25,
  16,
  18,
  20,
  17,
  30,
  27,
  25,
  2,
  23,
  22,
  20,
  21,
  9,
  22,
  23,
  19,
  8,
  12,
  21,
  31,
  23,
  13,
  4,
  11,
  9,
  32],
 'SC': [20,
  17,
  21,
  22,
  35,
  16,
  23,
  28,
  28,
  22,
  20,
  18,
  27,
  17,
  28,
  40,
  29,
  19,
  17,
  27,
  25,
  36,
  23,
  21,
  16,
  17,
  23,
  26],
 'OM': [12,
  20,
  26,
  14,
  14,
  25,
  65,
  13,
  22,
  2,
  27,
  16,
  15,
  20,
  8,
  14,
  16,
  15,
  13,
  5,
  21,
  19,
  24,
  31,
  9,
  16,
  16,
  23],
 'OC': [12,
  25,
  26,
  26,
  20,
  27,
  29,
  23,
  24,
  26,
  13,
  18,
  26,
  26,
  25,
  27,
  21,
  24,
  26,
  13,
  27,
  26,
  21,
  26,
  26,
  25,
  22,
  26]}

In [186]:
cond_bpress_dat['SM']

[array([124.24, 194.57]), array([113.1 , 136.02, 165.33, 198.41, 282.9 ])]

In [224]:
new_bpress_dat

{'sub-000': {'SM': [array([ 94.6 , 115.6 , 154.02, 221.83, 269.04]),
   array([ 27.81,  59.33,  74.99, 103.14, 126.75, 154.29, 167.91, 190.39,
          208.15, 225.51, 251.94, 276.34, 299.11])],
  'SC': [array([ 24.51,  47.41,  70.55,  96.8 , 124.51, 152.11, 182.5 , 214.31,
          252.61, 281.45]),
   array([ 21.66,  42.06,  63.67,  88.93, 118.27, 150.12, 187.09, 216.3 ,
          255.31, 299.71])],
  'OM': [array([ 26.15,  44.63,  71.6 ,  92.68, 110.85, 143.25, 203.31, 234.51,
          251.08, 269.36, 286.63, 296.68])],
  'OC': [array([ 24.08,  47.63,  71.26,  93.88, 116.83, 141.44, 166.61, 188.43,
          210.33, 231.93, 255.94, 288.24])]},
 'sub-001': {'SM': [array([  7.8 ,  30.85,  53.28,  67.01, 116.3 , 144.7 , 162.57, 178.74,
          200.63, 230.38, 247.79, 262.98, 290.36]),
   array([  2.58,  21.65,  27.5 ,  47.7 ,  72.38, 107.04, 145.3 , 167.35,
          193.33, 228.28, 261.19, 273.19])],
  'SC': [array([ 42.98,  85.26, 121.66, 161.35, 196.7 , 228.55, 254.42, 285.95])

In [163]:
run_bpress_arr

array([ 94.6 , 115.6 , 125.55, 154.02, 221.83, 269.04])

In [213]:
1-np.sum(cond_elim_bpress['SM']) / np.sum(cond_total_bpress['SM'])

0.14601018675721567

In [214]:
1-np.sum(cond_elim_bpress['OM']) / np.sum(cond_total_bpress['OM'])

0.1669394435351882

In [215]:
1-np.sum(cond_elim_bpress['OC']) / np.sum(cond_total_bpress['OC'])

0.003095975232198178

In [216]:
1-np.sum(cond_elim_bpress['SC']) / np.sum(cond_total_bpress['SC'])

0.021374045801526687

In [229]:
np.save(os.path.join(sav_bpress, "n28_4_conds_ts_press_ovrlpREMOV.npy"), new_bpress_dat)

In [227]:
new_bpress_dat['sub-000']

{'SM': [array([ 94.6 , 115.6 , 154.02, 221.83, 269.04]),
  array([ 27.81,  59.33,  74.99, 103.14, 126.75, 154.29, 167.91, 190.39,
         208.15, 225.51, 251.94, 276.34, 299.11])],
 'SC': [array([ 24.51,  47.41,  70.55,  96.8 , 124.51, 152.11, 182.5 , 214.31,
         252.61, 281.45]),
  array([ 21.66,  42.06,  63.67,  88.93, 118.27, 150.12, 187.09, 216.3 ,
         255.31, 299.71])],
 'OM': [array([ 26.15,  44.63,  71.6 ,  92.68, 110.85, 143.25, 203.31, 234.51,
         251.08, 269.36, 286.63, 296.68]),
  array([], dtype=float64)],
 'OC': [array([ 24.08,  47.63,  71.26,  93.88, 116.83, 141.44, 166.61, 188.43,
         210.33, 231.93, 255.94, 288.24]),
  array([], dtype=float64)]}

In [20]:
all_ = np.sum(cond_total_bpress['SM']) + np.sum(cond_total_bpress['OM']) + np.sum(cond_total_bpress['OC']) + np.sum(cond_total_bpress['SC'])
elim = np.sum(cond_elim_bpress['SM']) + np.sum(cond_elim_bpress['OM']) + np.sum(cond_elim_bpress['OC']) + np.sum(cond_elim_bpress['SC'])


In [21]:
elim

2359

In [23]:
all_

2566

In [24]:
1 - (elim / all_)

0.08067030397505848

In [144]:
new_bpress_dat

{'sub-021': {'OC': [array([ 23.57,  46.9 ,  71.27,  93.42, 116.05, 140.2 , 165.72, 189.35,
          212.07, 231.15, 256.45, 271.15]),
   array([ 21.06,  43.05,  62.29,  99.26, 131.41, 151.43, 169.94, 188.29,
          203.31, 219.21, 241.89, 265.86])]}}

In [143]:
cond_total_bpress

{'SM': [44, 44], 'SC': [38], 'OM': [25], 'OC': [26]}

In [117]:
all_bpress

{'sub-021': {'SM-1': 18,
  'SM-2': 26,
  'SC-1': 20,
  'SC-2': 18,
  'OM-1': 8,
  'OM-2': 17,
  'OC-1': 13,
  'OC-2': 13}}

In [70]:
len(sort_bpress[lab_indic_cor[lab][1]])

14

In [52]:
sub = ['sub-021']

In [82]:
lab = "SM"

In [46]:
sort_bpress[lab_indic_cor[lab][run]]

array([ 23.57,  46.73,  70.63,  94.03, 117.08, 140.65, 165.68, 188.75,
       216.85, 241.43, 264.7 , 292.83])

In [89]:
dif

11.45999999999998

In [113]:
 print(lab, "differences are:")
# merge both runs into one array
cond_diff = []
# mean / vars dict
new_bpress_dat = {}
run_bpress_dat = []
cond_bpress_dat = {}
count_tot_bpress = []
count_elim = []
for run in range(2):
    run_bpress_arr = sort_bpress[lab_indic_cor[lab][run]]

    # Insert zero for the start of the task
    #run_bpress_arr = np.insert(run_bpress_arr,0,0) 
    difs = np.zeros(len(run_bpress_arr)-1)
    for idx in range(1,len(run_bpress_arr)):
        # set the current index of the empty array (diffs) to the subtracted value
        # of the current bpress idx minus the bpress -1 index
        difs[idx-1] = run_bpress_arr[idx] - run_bpress_arr[idx-1]

    add_press = []
    last = False
    for idx, dif in enumerate(difs):
        #print(run_bpress_arr[idx])
        if dif < 12 and idx != 0:
            if last == True:
                # add that bpress
                add_press.append(run_bpress_arr[idx])
                # last remains true
            else:
                last=True
                continue
        else:
            add_press.append(run_bpress_arr[idx])
            last=False
    run_bpress_dat.append(np.asarray(add_press))
    print('num eliminated:', len(run_bpress_arr)-len(add_press), 'of', len(run_bpress_arr))
    count_tot_bpress.append(len(run_bpress_arr))
    count_elim.append(len(add_press))
cond_bpress_dat[lab] = run_bpress_dat
#new_bpress_dat[sub] = cond_bpress_dat
        
    

SM differences are:
num eliminated: 4 of 18
num eliminated: 10 of 26


In [115]:
cond_bpress_dat['SM']

[array([  9.3 ,  18.89,  39.2 ,  59.34,  74.14,  99.87, 112.43, 143.53,
        168.58, 185.18, 209.83, 226.1 , 247.38, 266.27]),
 array([  6.92,  26.99,  41.78,  65.72,  92.8 , 117.29, 147.8 , 153.25,
        171.08, 191.5 , 198.06, 211.  , 224.54, 245.06, 265.29, 277.88])]

6.92
15.73
26.99
41.78
58.5
65.72
86.78
92.8
108.07
117.29
136.74
147.8
153.25
171.08
185.42
191.5
198.06
211.0
224.54
236.79
245.06
257.35
265.29
277.88
294.43


In [104]:
add

[6.92,
 26.99,
 41.78,
 65.72,
 92.8,
 117.29,
 147.8,
 153.25,
 171.08,
 191.5,
 198.06,
 211.0,
 224.54,
 245.06,
 265.29,
 277.88]

In [106]:
difs

array([ 8.81, 11.26, 14.79, 16.72,  7.22, 21.06,  6.02, 15.27,  9.22,
       19.45, 11.06,  5.45, 17.83, 14.34,  6.08,  6.56, 12.94, 13.54,
       12.25,  8.27, 12.29,  7.94, 12.59, 16.55, 11.46])

In [94]:
sort_bpress[lab_indic_cor[lab][1]]

array([  6.92,  15.73,  26.99,  41.78,  58.5 ,  65.72,  86.78,  92.8 ,
       108.07, 117.29, 136.74, 147.8 , 153.25, 171.08, 185.42, 191.5 ,
       198.06, 211.  , 224.54, 236.79, 245.06, 257.35, 265.29, 277.88,
       294.43, 305.89])

In [105]:
len(add)

16

In [95]:
difs

array([ 8.81, 11.26, 14.79, 16.72,  7.22, 21.06,  6.02, 15.27,  9.22,
       19.45, 11.06,  5.45, 17.83, 14.34,  6.08,  6.56, 12.94, 13.54,
       12.25,  8.27, 12.29,  7.94, 12.59, 16.55, 11.46])

In [84]:
len(sort_bpress[lab_indic_cor[lab][0]])

18

In [85]:
len(sort_bpress[lab_indic_cor[lab][1]])

26

In [119]:
dat

,SM-1,SM-2,SC-1,SC-2,OM-1,OM-2,OC-1,OC-2
sub-000,6,13,10,10,14,0,12,0
sub-001,13,14,8,9,12,13,13,12
sub-002,8,8,12,9,20,13,13,13
sub-003,7,11,12,10,7,12,13,13
sub-004,5,15,17,18,9,7,14,7
sub-005,9,8,7,9,18,15,13,14
sub-006,17,24,12,11,30,46,16,13
sub-007,15,17,16,13,8,5,11,12
sub-008,11,15,14,17,14,15,12,12
sub-009,1,1,9,14,1,1,14,12


In [93]:
 print(lab, "differences are:")
# merge both runs into one array
cond_diff = []
# mean / vars dict
calcs = {}
new_bpress_dat = []
count_tot_bpress = []
count_elim = []
for run in range(2):
    run_bpress_arr = sort_bpress[lab_indic_cor[lab][run]]

    # Insert zero for the start of the task
    #run_bpress_arr = np.insert(run_bpress_arr,0,0) 
    difs = np.zeros(len(run_bpress_arr)-1)
    for idx in range(1,len(run_bpress_arr)):
        # set the current index of the empty array (diffs) to the subtracted value
        # of the current bpress idx minus the bpress -1 index
        difs[idx-1] = run_bpress_arr[idx] - run_bpress_arr[idx-1]
        
    less_than_12 = np.where(difs <12)[0]
    # Cuz first entry is zero, padded for differences
    new_bpress_dat.append(run_bpress_arr[1:][less_than_12])
    print('num eliminated:', len(run_bpress_arr[1:])-len(less_than_12))
    count_tot_bpress.append(len(run_bpress_arr[1:]))
    count_elim.append(len(less_than_12))
    #cond_diff.append(difs)
    
    
# now calculate means and differences 
# first merge two runs


SM differences are:
num eliminated: 13
num eliminated: 13


In [50]:
new_bpress_dat

[array([ 18.3 ,  40.1 ,  62.01,  82.81, 104.95, 125.63, 148.23, 170.01,
        193.55, 244.19]),
 array([ 23.57,  46.73,  70.63,  94.03, 117.08, 140.65, 188.75, 264.7 ])]

In [ ]:
    add_press = []
    last = False
    for idx, dif in enumerate(difs):
        print(sort_bpress[lab_indic_cor[lab][run]][idx])
        if dif < 12 and idx != 0:
            if last == True:
                # add that bpress
                add_press.append(sort_bpress[lab_indic_cor[lab][run]][idx])
                # last remains true
            else:
                last=True
                continue
        else:
            add_press.append(sort_bpress[lab_indic_cor[lab][run]][idx])
            last=False
    new_bpress_dat.append(add_press)
    print('num eliminated:', len(run_bpress_arr)-len(add_press))
    count_tot_bpress.append(len(run_bpress_arr))
    count_elim.append(len(add_press))

In [40]:
x = np.where(cond_diff[0] <24)[0]

In [44]:
run_bpress_arr[1:][x]

array([ 23.57,  46.73,  70.63,  94.03, 117.08, 140.65, 165.68, 188.75,
       216.85, 264.7 ])

In [47]:
cond_diff

[array([18.3 , 21.8 , 21.91, 20.8 , 22.14, 20.68, 22.6 , 21.78, 23.54,
        29.48, 21.16, 29.2 , 31.47]),
 array([23.57, 23.16, 23.9 , 23.4 , 23.05, 23.57, 25.03, 23.07, 28.1 ,
        24.58, 23.27, 28.13])]

In [14]:
all_bpress = {}
for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    if sub == 'sub-002': behav = behav.fillna("-1")
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "ts_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor)   
    sub_bpress_info = {}
    bpress_dat = {}
    summer = {}
    for lab in lab_indic_cor:
        ### SUM CUNT ###
        # Get the first, second index of the each condition #
        run1 = sort_bpress[lab_indic_cor[lab][0]]
        run2 = sort_bpress[lab_indic_cor[lab][1]]
        summer[lab+"-1"] = len(sort_bpress[lab_indic_cor[lab][0]])
        summer[lab+"-2"] = len(sort_bpress[lab_indic_cor[lab][1]])
        print("the", lab,"count is", mn)

    all_bpress[sub] = summer
    print()
print('el fin biotches')

BEGIN:  sub-000
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  14
hi
the SM count is [ 6 13]
the SC count is [10 10]
the OM count is [14  0]
the OC count is [12  0]

BEGIN:  sub-001
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  8
rowval: 7 colval: 4 dfra

/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


 Sum bpress:  7
the SM count is [ 5 15]
the SC count is [17 18]
the OM count is [9 7]
the OC count is [14  7]

BEGIN:  sub-005
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  18
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  7
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  9
rowval: 7 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  8
rowval: 8 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  9
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  15
rowval: 6 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress

SC Sum bpress:  8
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 8 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  7
the SM count is [4 4]
the SC count is [9 8]
the OM count is [8 7]
the OC count is [13 13]

BEGIN:  sub-019
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1']
SM Sum bpress:  4
rowval: 4 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  14
rowval: 5 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  14
rowval: 6 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  8
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
hi
rowval: 7 colval: 10 dframe val: ['1']
OC Sum bpress:  1
rowval: 8 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1']
OM Sum bpress:  6
the SM cou

In [23]:
sort_bpress[lab_indic_cor[lab][0]]

array([ 18.3 ,  40.1 ,  62.01,  82.81, 104.95, 125.63, 148.23, 170.01,
       193.55, 223.03, 244.19, 273.39, 304.86])

In [15]:
dat= pd.DataFrame(all_bpress).T

In [28]:
list(sort_bpress[lab_indic_cor[lab][0]]) + list(sort_bpress[lab_indic_cor[lab][1]])

[18.3,
 40.1,
 62.01,
 82.81,
 104.95,
 125.63,
 148.23,
 170.01,
 193.55,
 223.03,
 244.19,
 273.39,
 304.86,
 23.57,
 46.73,
 70.63,
 94.03,
 117.08,
 140.65,
 165.68,
 188.75,
 216.85,
 241.43,
 264.7,
 292.83]

In [72]:
#sub_list = ['sub-021']

In [73]:
#dat.to_csv('n28_bpress_counts.csv',index=False,sep='\t',mode='a')
#!mv n28_bpress_counts.csv /jukebox/graziano/coolCatIsaac/ATM/data/behavioral

# BPRESS ONSETS # 

In [74]:
all_bpress = {}
for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    if sub == 'sub-002': behav = behav.fillna("-1")
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "ts_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor)   
    sub_bpress_info = {}
    bpress_dat = {}
    summer = {}
    for lab in lab_indic_cor:
        temp_arr = []
        ### SUM CUNT ###
        # Get the first, second index of the each condition #
        #summer[lab+"-1"] = sort_bpress[lab_indic_cor[lab][0]]
        #summer[lab+"-2"] = sort_bpress[lab_indic_cor[lab][1]]
        temp_arr.append(sort_bpress[lab_indic_cor[lab][0]])
        temp_arr.append(sort_bpress[lab_indic_cor[lab][1]])
        summer[lab] = temp_arr
    all_bpress[sub] = summer
    print()
print('el fin biotches')

BEGIN:  sub-021
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  8
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  18
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  20
rowval: 6 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  18
rowval: 7 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  26
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  17
rowval: 8 colval: 10

In [19]:
all_bpress['sub-022']

{'SM': [array([ 75.91,  97.45, 141.7 , 173.18, 224.04, 266.33, 279.56, 294.89,
         305.08]),
  array([ 62.57,  78.72,  84.85,  98.95, 109.27, 126.7 , 145.73, 152.74,
         170.84, 179.08, 192.2 , 206.73, 211.83, 221.29, 226.91, 236.98,
         260.68, 264.3 , 277.33, 288.61, 297.16, 304.43])],
 'SC': [array([ 22.03,  47.42,  74.46,  99.18, 127.43, 153.06, 180.13, 206.81,
         233.38, 256.23, 283.93, 307.47]),
  array([ 18.8 ,  40.5 ,  66.56,  92.4 , 121.55, 149.  , 178.53, 208.6 ,
         237.99, 265.35, 294.11])],
 'OM': [array([ 18.83,  39.98,  67.23,  83.  ,  85.53, 112.73, 136.96, 156.18,
         163.53, 188.7 , 221.93, 238.44, 267.59]),
  array([ 15.83,  19.22,  30.72,  44.87,  47.7 ,  80.98, 110.85, 126.28,
         152.17, 203.52, 234.38, 251.11, 269.65, 276.93, 286.5 , 299.81])],
 'OC': [array([ 18.17,  40.13,  62.3 ,  81.9 , 104.2 , 129.48, 151.3 , 178.5 ,
         208.15, 235.53, 290.21]),
  array([ 24.12,  46.53,  71.4 ,  92.72, 111.55, 147.28, 198.12, 220.75,

In [27]:
#np.save(os.path.join(sav_bpress, "n28_4_conds_tr_stamp.npy"), all_bpress)

In [113]:
!pwd

/mnt/cup/labs/graziano/coolCatIsaac/ATM/code/analysis/bpress


In [20]:
sav_bpress

'/jukebox/graziano/coolCatIsaac/ATM/data/work/results/bpress_GLM/behav'

In [28]:
sub_list = ['sub-001']

# nick and josh bpress separately # 

In [34]:
def bpress_arrays(dframe, sub_ses_labels, dtype):
    """
    Take in a column value (self or other) and count the number of presses for that particular run
    First get all self_btn info, add to an array. 
    Second, get all other info, add to an array
    All data is UNSORTED.
    args:
        cleaned subject dataframe
        dtype: tr_stamp = the tr when the button press occured
               ts_stamp = the time in seconds when the button press occured
               bin_arr = the binary array of when the button press occured (tr)
        
    return: 
        cond: unsorted array of labels (Self, then Other conds)
        Bpress: shape = 8 conditions by X TRs. 1s and 0s


    """
    if dtype == "tr_stamp":
        get_press = 1
    elif dtype == "ts_stamp":
        get_press= 2
    elif dtype == "bin_stamp":
        get_press= 0
    else:
        print("enter dtype arg correctly yo")
    
    df = []
    cond = []
    
    for col_ind, col_val in enumerate(dframe.columns):
        if col_val == "self_btn.keys":
            # For all Rows in the self column
            # if the row is of type string
            for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                #if type(row_val) is type("a"):
                if row_val != "-1":
                    # Add to condition array
                    # There are 10 rows, and 10 runs. 
                    # I could pull from the third column to get the string name
                    # Instead i pull from the label order in sub_ses, which corresponds to the
                    # row_inx
                    cond.append(sub_ses_labels[row_inx])
                    """
                    run_press takes in the button presses, then the timing info (+1)
                    returns: a list of button presses, 1s and 0s for each run - [0]
                            - OR the TR when the button press occured - [1]
                    """
                    print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                    bpress_info = dframe.iloc[row_inx,col_ind]
                    time_info = dframe.iloc[row_inx,col_ind+1]
                    df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                    tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                    print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
        elif col_val == "other_j_btn1.keys":
            if sub == 'sub-000' and col_val =="other_n_btn.keys":
                print('hi')
                df.append(np.array([]))
                df.append(np.array([]))
                cond.append('OM-N')
                cond.append('OC-N')
            else:
                for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                    if row_val == "None":
                        print("hi")
                        df.append(np.array([]))
                        cond.append(sub_ses_labels[row_inx])
                    elif row_val != "-1":
                        print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                        # Add to condition array
                        cond.append(sub_ses_labels[row_inx])
                        # append button press info
                        bpress_info = dframe.iloc[row_inx,col_ind]
                        time_info = dframe.iloc[row_inx,col_ind+1]
                        df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                        tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                        print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
        elif col_val =="other_n_btn.keys":
            if sub == 'sub-000' and col_val =="other_n_btn.keys":
                print('hi')
                df.append(np.array([]))
                df.append(np.array([]))
                cond.append('OM')
                cond.append('OC')
            else:
                for row_inx, row_val in enumerate(dframe.iloc[0:,col_ind]):
                    if row_val == "None":
                        print("hi")
                        df.append(np.array([]))
                        cond.append(sub_ses_labels[row_inx])
                    elif row_val != "-1":
                        print("rowval:",row_inx, "colval:",col_ind, "dframe val:", dframe.iloc[row_inx,col_ind])
                        # Add to condition array
                        cond.append(sub_ses_labels[row_inx])
                        print(sub_ses_labels[row_inx], "nick", row_inx)
                        # append button press info
                        bpress_info = dframe.iloc[row_inx,col_ind]
                        time_info = dframe.iloc[row_inx,col_ind+1]
                        df.append(np.asarray(run_press(bpress_info, time_info)[get_press]).flatten())
                        tem = sum(run_press(dframe.iloc[row_inx,col_ind],dframe.iloc[row_inx,col_ind+1])[0])
                        print(sub_ses_labels[row_inx], "Sum bpress: ", tem)
    return np.asarray(df), cond



In [35]:
cond

['SM', 'SC', 'SC', 'SM', 'OM', 'OC', 'OM', 'OC']

In [36]:
all_bpress = {}
for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    if sub == 'sub-002': behav = behav.fillna("-1")
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "tr_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor)   
    sub_bpress_info = {}
    bpress_dat = {}
    summer = {}
    for lab in lab_indic_cor:
        temp_arr = []
        ### SUM CUNT ###
        # Get the first, second index of the each condition #
        #summer[lab+"-1"] = sort_bpress[lab_indic_cor[lab][0]]
        #summer[lab+"-2"] = sort_bpress[lab_indic_cor[lab][1]]
        temp_arr.append(sort_bpress[lab_indic_cor[lab][0]])
        temp_arr.append(sort_bpress[lab_indic_cor[lab][1]])
        summer[lab] = temp_arr
    all_bpress[sub] = summer
    print()
print('el fin biotches')

BEGIN:  sub-001
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  8
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  9
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  14
rowval: 2 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM nick 2
OM Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC nick 4
OC Sum bpress:  13
rowval: 5 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  13
rowval: 6 colval: 10 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12

el fin biotches


In [21]:
sub = ['sub-025']

In [20]:
lab_indic_cor

{'SM': [3, 7], 'SC': [0, 6], 'OM': [2, 4], 'OC': [1, 5]}

In [22]:
all_bpress = {}
for sub in sub_list:
    print("BEGIN: ", sub)
    # Load Behavioral
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    label = behav.iloc[:,1]
    sub_ses_labels = label_lists(label,num_trs)
    # Get all button press data in array form, 1s and 0s. returns an array 8x209 
    # Other options: tr_stamp, bin_stamp, ts_stamp
    unsort_bpress, cond = bpress_arrays(behav, sub_ses_labels[0], "ts_stamp")
    # Find condition indexes for the unsorted + sorted data
    # Eliminate rest conditions from orignal run labeled data
    lab_indic_cor = find_cond_index(sub_ses_labels[0][1:-1])
    lab_indic_inc = find_cond_index(cond)
    # Returns 8x209 dataframe of 1s and 0s, sorted correctly by when the run occured
    sort_bpress = sort_bpress_data_onset(unsort_bpress,lab_indic_inc, lab_indic_cor)   
    sub_bpress_info = {}
    bpress_dat = {}
    for lab in lab_indic_cor:
        ### SUM CUNT ###
        # Get the first, second index of the each condition #
        mn = np.hstack((len(sort_bpress[lab_indic_cor[lab][0]]),len(sort_bpress[lab_indic_cor[lab][1]])))
        #summer[lab] = np.mean(mn)
        print("the", lab,"count is", mn)
        #for idx in lab_indic_inc[lab]:
        ### Interval Info ###
        # Create empty array with a length of the number of button presses
        # minus one cuz we subtract pairs in the array from one another
        # For each run in the condition
        print(lab, "differences are:")
        # merge both runs into one array
        cond_diff = []
        # mean / vars dict
        calcs = {}
        for run in range(2):
            run_bpress_arr = sort_bpress[lab_indic_cor[lab][run]]
            # Insert zero for the start of the task
            run_bpress_arr = np.insert(run_bpress_arr,0,0) 
            difs = np.zeros(len(run_bpress_arr)-1)
            for idx in range(1,len(run_bpress_arr)):
                # set the current index of the empty array (diffs) to the subtracted value
                # of the current bpress idx minus the bpress -1 index
                difs[idx-1] = run_bpress_arr[idx] - run_bpress_arr[idx-1]
            print(difs)
            cond_diff.append(difs)
        # now calculate means and differences 
        # first merge two runs
        merg = np.concatenate(cond_diff)
        calcs = {'mean_int': np.mean(merg), 'sd_int': stat.stdev(merg), 'av_press': np.mean(mn)}
        sub_bpress_info[lab] = calcs
    all_bpress[sub] = sub_bpress_info
    print()
print('el fin biotches')

BEGIN:  sub-000
rowval: 1 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1']
SM Sum bpress:  6
rowval: 2 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 7 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SC Sum bpress:  10
rowval: 8 colval: 4 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
SM Sum bpress:  13
rowval: 3 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OC Sum bpress:  12
rowval: 4 colval: 7 dframe val: ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
OM Sum bpress:  14
hi
the SM count is [ 6 13]
SM differences are:
[94.6  21.    9.95 28.47 67.81 47.21]
[27.81 31.52 15.66 28.15 23.61 27.54 13.62 22.48 17.76 17.36 26.43 24.4
 22.77]
the SC count is [10 10]
SC differences are:
[24.51 22.9  23.14 26.25 27.71 27.6  30.39 31.81 38.3  28.84]
[21.66 20.4  21.61 25.26 29.34 31.85 36.97 29.21 39.01 44.4 ]
the OM count is

AttributeError: 'float' object has no attribute 'split'

In [15]:
merg

array([18.3 , 21.8 , 21.91, 20.8 , 22.14, 20.68, 22.6 , 21.78, 23.54,
       29.48, 21.16, 29.2 , 31.47, 23.57, 23.16, 23.9 , 23.4 , 23.05,
       23.57, 25.03, 23.07, 28.1 , 24.58, 23.27, 28.13])

In [30]:
lab_indic_inc

{'SM': [2, 5], 'SC': [3, 4], 'OM': [0, 6], 'OC': [1, 7]}

In [31]:
lab_indic_cor

{'SM': [2, 6], 'SC': [3, 5], 'OM': [0, 4], 'OC': [1, 7]}

In [21]:
sort_bpress

[array([  7.8 ,  30.85,  53.28,  67.01, 116.3 , 144.7 , 162.57, 178.74,
        200.63, 230.38, 247.79, 262.98, 290.36]),
 array([ 13.35,  39.75,  75.18,  82.57,  85.  , 112.55, 155.96, 188.75,
        200.98, 221.06, 222.58, 238.51]),
 array([ 42.98,  85.26, 121.66, 161.35, 196.7 , 228.55, 254.42, 285.95]),
 array([ 18.3 ,  40.1 ,  62.01,  82.81, 104.95, 125.63, 148.23, 170.01,
        193.55, 223.03, 244.19, 273.39, 304.86]),
 array([ 34.31,  80.84, 101.98, 107.63, 112.46, 153.61, 156.56, 203.79,
        234.54, 269.91, 275.49, 287.09, 294.51]),
 array([ 23.57,  46.73,  70.63,  94.03, 117.08, 140.65, 165.68, 188.75,
        216.85, 241.43, 264.7 , 292.83]),
 array([ 40.25,  74.61, 102.55, 132.45, 164.88, 197.5 , 226.1 , 254.84,
        288.92]),
 array([  2.58,  12.98,  21.65,  27.5 ,  47.7 ,  72.38,  76.05, 107.04,
        145.3 , 167.35, 193.33, 228.28, 261.19, 273.19])]

In [32]:
df = pd.DataFrame(all_bpress)
df = df.T

In [33]:
chek = pd.DataFrame.from_dict({(i,j): df[i][j] 
                           for i in df.keys() 
                           for j in df[i].keys()})


In [16]:

###### OG ####
chek['SM'].T

,mean_int,sd_int,av_press
sub-001,23.713333,7.213056,13.0
sub-003,27.786667,10.705369,10.0
sub-004,24.395500,16.217508,11.0
sub-005,22.234545,9.439308,12.0
sub-006,12.533111,8.911257,23.5
sub-007,22.349167,8.096207,13.0
sub-008,25.963333,10.028908,11.5
sub-011,32.323750,19.777697,9.0
sub-012,20.680833,8.702605,13.0
sub-013,23.621304,4.572161,12.5


In [34]:
chek['SM'].T

,mean_int,sd_int,av_press
sub-001,20.872222,11.222561,13.5
sub-003,32.580000,13.151676,9.0
sub-004,29.767000,20.285543,10.0
sub-005,34.494706,14.326242,8.5
sub-006,14.751220,9.670633,20.5
sub-007,18.650938,6.687291,16.0
sub-008,23.031538,10.604360,13.0
sub-011,22.435185,18.755129,13.5
sub-012,23.760000,15.538411,11.5
sub-013,25.627391,13.851810,11.5


In [35]:
sm = chek['SM'].T

In [36]:
# mean bpress
np.mean(sm.values[:,2])

12.294117647058824

In [37]:
# std bpress
stat.stdev(sm.values[:,2])

4.623509298713924

In [38]:
# mean interval
np.mean(sm.values[:,0])

26.96618247729081

In [39]:
# sd interval
np.median(sm.values[:,1])

13.851809930110973

In [40]:
# sd interval
np.mean(sm.values[:,1])

14.446007607890174

In [80]:
dict0 = {'sum':15, 'mean_dir': 10.2, 'var':5}

In [84]:
dict1 = {'SM':np.asarray([15,10.3, 5]), 'OM':np.asarray([15,10.3, 5])}
dict1 = {'SM':dict0, 'OM':dict0}
dict2 = {'sub-001': dict1, 'sub-002': dict1,'sub-003': dict1,'sub-004': dict1}

In [85]:
df = pd.DataFrame(dict2)
df = df.T

In [99]:
chek = pd.DataFrame.from_dict({(i,j): df[i][j] 
                           for i in df.keys() 
                           for j in df[i].keys()})


In [92]:
chek['sum']

SM  sub-001    15
    sub-002    15
    sub-003    15
    sub-004    15
OM  sub-001    15
    sub-002    15
    sub-003    15
    sub-004    15
Name: sum, dtype: int64

In [102]:
chek['SM'].T

,sum,mean_dir,var
sub-001,15.0,10.2,5.0
sub-002,15.0,10.2,5.0
sub-003,15.0,10.2,5.0
sub-004,15.0,10.2,5.0


In [98]:
chek.set_names(['cond', 'sub'], inplace = True)

AttributeError: 'DataFrame' object has no attribute 'set_names'

In [94]:
chek.index

MultiIndex([('SM', 'sub-001'),
            ('SM', 'sub-002'),
            ('SM', 'sub-003'),
            ('SM', 'sub-004'),
            ('OM', 'sub-001'),
            ('OM', 'sub-002'),
            ('OM', 'sub-003'),
            ('OM', 'sub-004')],
           )

In [96]:
chek[chek.index['SM']]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [52]:
df.T

,SM,OM
sub-001,"[15.0, 10.3, 5.0]","[15.0, 10.3, 5.0]"
sub-002,"[15.0, 10.3, 5.0]","[15.0, 10.3, 5.0]"
sub-003,"[15.0, 10.3, 5.0]","[15.0, 10.3, 5.0]"
sub-004,"[15.0, 10.3, 5.0]","[15.0, 10.3, 5.0]"


In [56]:
ew = np.asarray(df.T)

In [59]:
ew

array([[array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])],
       [array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])],
       [array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])],
       [array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])]],
      dtype=object)

In [60]:
ew.flatten()

array([array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])], dtype=object)

In [61]:
# Get SM
ew = ew[:,1]

In [63]:
ew.flatten()

array([array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])], dtype=object)

In [64]:
ew

array([array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ]),
       array([15. , 10.3,  5. ]), array([15. , 10.3,  5. ])], dtype=object)

In [62]:
sort_bpress[7]

array([ 21.06,  43.05,  62.29,  99.26, 131.41, 151.43, 169.94, 188.29,
       203.31, 219.21, 241.89, 265.86, 283.69])

In [56]:
sort_bpress[0]

array([ 34.81,  43.16, 111.25, 128.11, 162.16, 237.76, 280.66, 294.59])

In [57]:
print(difs)

[ 8.35 68.09 16.86 34.05 75.6  42.9  13.93]


In [50]:
sort_bpress[0][0]

34.81

In [51]:
sort_bpress[0][1] - sort_bpress[0][0]

8.349999999999994

In [25]:
len(sort_bpress[5])

14

In [18]:
## list 
#Inner loop - across runs #
count_bpress = []
#outisde loop - across subjects
# COUNT BPRESS
sub_bpress = []
# CONDITION LABEL
cond_arr = []

In [60]:
cond_arr.append(lab_indic_inc)

In [61]:
cond_arr

[{'SM': [0, 3], 'SC': [1, 2], 'OM': [5, 7], 'OC': [4, 6]},
 {'SM': [0, 3], 'SC': [1, 2], 'OM': [5, 7], 'OC': [4, 6]},
 {'SM': [0, 3], 'SC': [1, 2], 'OM': [5, 7], 'OC': [4, 6]}]

In [47]:
count_bpress.append(len(sort_bpress[0]))

In [48]:
count_bpress

[1, 1, 1, 1, 1]

In [51]:
sub_bpress.append(count_bpress)

In [52]:
sub_bpress

[[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]

In [93]:
sub_ses_labels[0]

['Re', 'SC', 'SM', 'OM', 'OC', 'SC', 'OM', 'SM', 'OC', 'Re']

In [100]:
a[1]

['Re', 'SC', 'SM', 'OM', 'OC', 'SC', 'OM', 'SM', 'OC', 'Re']

In [ ]:
sub_ses_labels[0]

In [102]:
b.append(len(sort_bpress[0]))

12

In [105]:
for x in range(1,12):
    print(x)

1
2
3
4
5
6
7
8
9
10
11


In [38]:
lab_indic_cor

{'SM': [1, 6], 'SC': [0, 4], 'OM': [2, 5], 'OC': [3, 7]}

In [39]:
behav

,Index,blk_name,begin_scan_btn.rt,begin_scan_btn.started,self_btn.keys,self_btn.rt,self_btn.started,other_n_btn.keys,other_n_btn.rt,other_n_btn.started,other_j_btn1.keys,other_j_btn1.rt,other_j_btn1.started
0,1,Rest_title.started,-1,-1.000000,-1,-1,-1.000000,-1,-1,-1.000000,-1,-1,-1.000000
1,5,MC_self_title.started,[1.4852273670258],420.648345,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[20.37084077997133, 45.95158576697577, 71.5679...",422.143973,-1,-1,-1.000000,-1,-1,-1.000000
2,7,MM_self_title.started,[1.4881828770739958],771.289100,"['1', '1', '1', '1', '1', '1', '1']","[51.46822209702805, 86.65644877194427, 125.071...",772.788834,-1,-1,-1.000000,-1,-1,-1.000000
3,9,MM_other_title.started,[1.4850854789838195],1123.567894,-1,-1,-1.000000,"['1', '1', '1', '1', '1', '1', '1']","[34.73545014404226, 39.43493033305276, 69.2674...",1125.501183,-1,-1,-1.000000
4,11,MC_other_title.started,[1.4840696200262755],1477.330291,-1,-1,-1.000000,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[12.868375902064145, 34.21847029705532, 57.018...",1479.062508,-1,-1,-1.000000
5,14,MC_self_title.started,[1.4890675380593166],1839.341232,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[26.03921395598445, 55.23933592799585, 82.4899...",1840.840920,-1,-1,-1.000000,-1,-1,-1.000000
6,16,MM_other_title.started,[1.484316918067634],2188.536311,-1,-1,-1.000000,-1,-1,-1.000000,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[86.63426328299101, 99.4346656659618, 110.7354...",2190.269583
7,18,MM_self_title.started,[1.4852906489977613],2535.515794,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[28.289827693952248, 47.468303015921265, 73.31...",2537.014617,-1,-1,-1.000000,-1,-1,-1.000000
8,20,MC_other_title.started,[1.4857432399876416],2880.727126,-1,-1,-1.000000,-1,-1,-1.000000,"['1', '1', '1', '1', '1', '1', '1', '1', '1', ...","[23.184997824020684, 46.602244504960254, 71.33...",2882.476853
9,23,Rest_title.started,-1,-1.000000,-1,-1,-1.000000,-1,-1,-1.000000,-1,-1,-1.000000


In [40]:
np.save(sav_work + sub + "bpress_rundata", sort_bpress, allow_pickle=True)


/jukebox/pkgs/PYGER/base/envs/0.11.0/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
# Example visualization code -- BROKEN, will not work

In [ ]:
### Self - Run 2 ###
num_TR = 209
# Create a sequence of timepoints that a TR occurred on
time_points = list(range(1,num_TR+1)) #np.arange(0, (vdc_TRs_run - 1) * 1.5 + 1, 1.5)

# Plot the data
f, ax = plt.subplots(1,1, figsize = (12,5))
#ax.plot(time_points, stim_label_TR[0:vdc_TRs_run, 0], c='orange')
ax.plot(time_points, sc_press_2, c='orange')
ax.plot(sm_press_2, c='teal')

plt.legend(["Self Count", "Self Monitor"])
ax.set_ylabel('Button Press')
ax.set_xlabel('TRs')
ax.set_title("Self Run 2")